In [1]:
import pickle
import numpy as np

In [2]:
with open('train_qa.txt', 'rb') as f:
    train_data = pickle.load(f)

In [3]:
with open('test_qa.txt', 'rb') as f:
    test_data = pickle.load(f)

In [4]:
type(test_data)

list

In [5]:
type(train_data)

list

In [6]:
len(train_data)

10000

In [7]:
len(test_data)

1000

In [8]:
# Story
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [9]:
# Question
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [10]:
# Answer
train_data[0][2]

'no'

In [11]:
all_data = test_data + train_data

In [12]:
vocab = set()

for story, question, answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [13]:
vocab.add('no')

In [14]:
vocab.add('yes')

In [15]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [16]:
vocab_len = len(vocab) + 1

In [17]:
vocab_len

38

In [18]:
# Longest story
all_story_lens = [len(data[0]) for data in all_data]

In [19]:
max_story_len = max(all_story_lens)

In [20]:
max_story_len

156

In [21]:
max_question_len = max([len(data[1]) for data in all_data ])

In [22]:
max_question_len

6

In [23]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [24]:
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(vocab)

In [25]:
tokenizer.word_index

{'football': 1,
 'daniel': 2,
 'mary': 3,
 'down': 4,
 'left': 5,
 '.': 6,
 'milk': 7,
 'grabbed': 8,
 'travelled': 9,
 'dropped': 10,
 'bedroom': 11,
 'yes': 12,
 'is': 13,
 'garden': 14,
 'bathroom': 15,
 '?': 16,
 'office': 17,
 'to': 18,
 'hallway': 19,
 'no': 20,
 'in': 21,
 'back': 22,
 'moved': 23,
 'journeyed': 24,
 'picked': 25,
 'put': 26,
 'there': 27,
 'up': 28,
 'the': 29,
 'john': 30,
 'went': 31,
 'apple': 32,
 'got': 33,
 'sandra': 34,
 'took': 35,
 'discarded': 36,
 'kitchen': 37}

In [26]:
train_story_text = []
train_question_text = []
train_answers = []

In [27]:
for story, question, answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [29]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [30]:
len(train_story_seq)

10000

In [31]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len, max_question_len=max_question_len):
    # Stories
    X = []
    # Questions
    Xq = []
    # Y Correct answer
    Y = []
    
    for story, question, answer in data:
        # for each story
        # [23, 14....]
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in question]
        
        y = np.zeros(len(word_index) + 1)
        
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    return (pad_sequences(X, maxlen=max_story_len), pad_sequences(Xq, maxlen=max_question_len), np.array(Y))
        

In [32]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [33]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)